## QCLab: 2-Qubit Variational Quantum Classifier (VQC)

A **Variational Quantum Classifier (VQC)** is a hybrid quantum-classical model for binary classification. It:
- **Encodes features** as quantum states via single-qubit rotations.
- **Applies a variational layer** of trainable gates with entanglement to shape a decision boundary.
- **Measures** to obtain class probabilities.
- **Uses a classical optimizer** to tune parameters by minimizing a loss over labeled data.

![2-qubit-VQC-classifier](images/2-qubit-VQC-classifier.png)


Prediction uses repeated measurements. For a single measured qubit:
$$
P(1)=\frac{\text{counts}(1)}{\text{shots}},\qquad P(0)=1-P(1).
$$

A hard label is assigned by
$$
\hat{y}=\begin{cases}
1 & \text{if } P(1)>0.5,\\
0 & \text{otherwise.}
\end{cases}
$$

Training minimizes the L2 loss over a dataset $\{(x_i,y_i)\}_{i=1}^N$:
$$
L(\theta)=\frac{1}{N}\sum_{i=1}^{N}\big(P_\theta(x_i)-y_i\big)^2,
$$
where $P_\theta(x_i)$ is the model’s predicted probability of class 1 for input $x_i$.

**Note**: In general literature, VQC may also stand for *Variational Quantum Circuit*.  

---

### Task

Build a **Variational Quantum Classifier (VQC)** to help a drone decide where to land based on two sensor readings.  
- Low sensor values represent safe, flat terrain (**label 0**).  
- High values indicate hazardous, uneven ground (**label 1**).  

To enable the classifier to learn how to separate safe and hazardous regions and guide the drone’s landing decisions, training and test datasets are provided.

### Expected Output

- Optimized VQC parameters after training.
- Predicted labels for the test set with the associated probabilities $P(1)$.
- A decision-boundary plot over the 2D feature space highlighting safe vs. hazardous regions.
- A circuit diagram of the trained 2-qubit VQC.

### Experimentation

- Modify or expand the training data and observe how the decision boundary shifts.
- Adjust the variational layer depth or entanglement pattern and compare accuracy.
- Try alternative losses (e.g., L1) or optimizers and compare convergence.
- Vary the number of shots to study sampling effects on predictions.


In [ ]:
# ====================================================
# QCLab: 2-Qubit Variational Quantum Classifier (VQC)
# <QC|CT> qcict.org
# ====================================================

from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numpy as np


# -- build VQC circuit --
def vqc(x, theta):
    qc = QuantumCircuit(2, 1)
    
    # feature encoding
    qc.ry(x[0], 0)
    qc.ry(x[1], 1)

    # variational layer
    qc.ry(theta[0], 0)
    qc.ry(theta[1], 1)
    qc.cx(0, 1)

    # measurement
    qc.measure(0, 0)
    return qc

# -- simulate and return label + probability of '1' --
def predict_label(x, theta, shots=1000):
    qc = vqc(x, theta)
    simulator = AerSimulator()
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    p = counts.get('1', 0) / shots
    return 1 if p > 0.5 else 0, p

# -- compute L2 loss on training set --
def l2_loss(theta):
    total = 0
    for x, y in train_data:
        _, p1 = predict_label(x, theta)
        total += (p1 - y)**2
    return total / len(train_data)
    
# -----------------------------------------------
#                main program
# -----------------------------------------------

# -- training dataset: ([x0, x1], label) --
train_data = [
    ([0.1, 0.2], 0),
    ([0.2, 0.4], 0),
    ([0.9, 0.8], 1),
    ([0.8, 0.9], 1)
]
# -- test dataset --
test_data = [
    ([0.05, 0.1], 0),
    ([0.95, 0.85], 1)
]

# -- initialize parameters and optimize --
theta0 = np.random.uniform(0, 2 * np.pi, 2)
result = minimize(l2_loss, theta0, method='COBYLA')
theta_opt = result.x

# -- evaluate on test set --
print("\nTest results:")
for x, y in test_data:
    y_pred, prob = predict_label(x, theta_opt)
    print(f"Input: {x} → Predicted: {y_pred} (p='1'={prob:.2f}), Actual: {y}")

# -- plot the VQC --
qc = vqc(test_data[0][0], theta_opt)
display(circuit_drawer(qc, style="bw", output="mpl"))

# -- visualize decision boundary --
x_vals = np.linspace(0, 1.2, 30)
y_vals = np.linspace(0, 1.2, 30)
xx, yy = np.meshgrid(x_vals, y_vals)
zz = np.zeros_like(xx)

for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        _, p1 = predict_label([xx[i, j], yy[i, j]], theta_opt, shots=200)
        zz[i, j] = p1
plt.figure(figsize=(10, 4))
plt.contourf(xx,yy,zz,levels=[0,0.5,1],alpha=0.6,colors=["#FFA07A","#87CEFA"])
plt.colorbar(label="P(pred = 1)")

# plot training points
for x, y in train_data:
    if y == 0:
        plt.plot(x[0], x[1], 'ro')
    else:
        plt.plot(x[0], x[1], 'bo')

# plot test points
for x, y in test_data:
    if y == 0:
        plt.plot(x[0], x[1], 'r^')
    else:
        plt.plot(x[0], x[1], 'b^')

plt.title("VQC Decision Boundary")
plt.xlabel("Feature x0")
plt.ylabel("Feature x1")
plt.grid(True)
plt.show()
